#MSM of the alanine dipeptide
Here we run through most of the things that can be done with this package using a simple two-state model. There are more sophisticated examples that enable for further possibilities. 
First we import a number of libraries we will need as we run this example

In [1]:
%load_ext autoreload
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


###Discretizing the trajectory
We start loading the simulation data using the `trajectory` module. For this we use the external library [`MDtraj`](http://mdtraj.org), which contains all sorts of methods for parsing and calculating interestign properties of our time-series data.

In [2]:
import mdtraj as md
from mastermsm.trajectory import traj

In [3]:
tr = traj.TimeSeries(top='data/alaTB.gro', traj=['data/protein_only.xtc'])
print tr.mdt

<mdtraj.Trajectory with 10003 frames, 19 atoms, 3 residues, and unitcells>


In [4]:
tr.discretize(states=['A', 'E'])
tr.find_keys()

In [5]:
from mastermsm.msm import msm
msm_alaTB = msm.SuperMSM([tr])


 Building MSM from 
[['data/protein_only.xtc']]
     # states: 2


In [6]:
for i in [1, 2, 5, 7, 10, 20, 50, 100]:
    msm_alaTB.do_msm(i)
    msm_alaTB.msms[i].do_trans()
    #msm_alaTB.msms[i].boots(plot=False)

The method produces acceptable solutions for short lag times although the result rapidly diverges from the transition matrix relaxation time at long lag times. The rate matrix elements have the following values:

In [7]:
msm_alaTB.msms[1].do_rate()
msm_alaTB.msms[1].rate

array([[-0.01251929,  0.01726619],
       [ 0.01251929, -0.01726619]])

Alternatively, we can use the lifetime based method, where we use branching probabilities and average lifetimes. The rate is almost identical to that from the Taylor expansion.

In [8]:
msm_alaTB.do_lbrate()
print msm_alaTB.lbrate

[[-0.01251929  0.01730769]
 [ 0.01251929 -0.01730769]]


Finally, we use Buchete and Hummer's maximum likelihood propagator-based method, which is also implemented in MasterMSM.

In [9]:
msm_alaTB.msms[1].do_rate(method='MLPB')

[[-7.15074706  4.20446054]
 [ 7.15074706 -4.20446054]]
[ 0.5830417  0.4169583]
[[-7.15074706  9.99904223]
 [ 7.15074706 -4.20446054]]
36427.6831472


ValueError: low >= high